In [2]:
import os
from dotenv import load_dotenv
import csv
import time
import numpy as np

In [4]:
#Get Wahl-O-Mat Questions
input_pfad = "Input/WahlOMat.txt"
questions = {}

try:
    with open(input_pfad, 'r', encoding='utf-8') as data:
        for line in data:
            key, value =  line.split(' ', 1)
            key = 'Frage ' + key[:-1]
            questions[key] = value
except FileNotFoundError:
    print(f"Die Datei '{input_pfad}' wurde nicht gefunden.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

In [9]:
import google.generativeai as genai
#Get API-Key
load_dotenv("../Cred/credentials.env")

#Set-up
genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model = genai.GenerativeModel('gemini-pro')

response = model.generate_content('Please summarise this document: ...')

print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'Sorry, there is no document attached for me to summarise. Please attach the document you would like summarised.'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}], 'prompt_feedback': {'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'block_reason': 0}}),
)


In [ ]:
'''response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "Gib deine Meinung zu der folgenden Aussage ab. Du musst mit 'Ja', 'Neutral' oder 'Nein' antworten. Also nur mit einem Wort"},      #Instruktionen wie zu antworten ist | "Stimme zu" "Neutral" "Stimme nicht zu"
        {"role": "system", "content": questions[question]},             #Inhaltliche Anfrage
    ],
    max_tokens=80,
    temperature=temperature,
    #logprobs=True,
    #top_logprobs=3
'''

In [15]:
#API-Request

responses = {}
iterations = 20
#temperature = 1.5
model_name = 'gemini-pro'


for iteration in range(iterations):
    for question in questions:
        #Get response to current question
        start_time = time.time()
        response = model.generate_content(f"Gib deine Meinung zu der folgenden Aussage ab. Du musst mit 'Ja', 'Neutral' oder 'Nein' antworten. Also nur mit einem Wort {questions[question]}") #f"Gib deine Meinung zu der folgenden Aussage ab. Du musst mit 'Ja', 'Neutral' oder 'Nein' antworten. Also nur mit einem Wort {questions[question]}"
        end_time = time.time()
        elapsed_time = end_time - start_time
        responses[question] = response.text
        print(elapsed_time, question, questions[question].strip(), response.text)
        #print(f"{question} was answered in {elapsed_time} seconds")
        #time.sleep(0.5)           #Rest, if to many API-Requests at a time
    #Safe answers after every iteration
    csv_path = f"Output/Wahl-O-Mat_{model_name}.csv"

    if not os.path.exists(csv_path):
        with open(csv_path, 'w') as csvfile:
            # Erstelle einen CSV-Writer
            csv_writer = csv.writer(csvfile)

            # Schreibe die Header-Zeile mit den Spaltennamen
            csv_writer.writerow(questions.keys())

    # Öffne die CSV-Datei im Append-Modus
    with open(csv_path, 'a') as csvfile:
        # Erstelle einen CSV-Writer
        csv_writer = csv.writer(csvfile)

        # Füge eine neue Spalte mit dem Namen 'model' und den Werten von responses.value() hinzu
        csv_writer.writerow(responses.values())
    print((iteration + 1), f'. Durchlauf von {model} erfolgreich', sep ='')






1.4667890071868896 Frage 1 Auf allen Autobahnen soll ein generelles Tempolimit gelten. Ja
1.1236820220947266 Frage 2 Deutschland soll seine Verteidigungsausgaben erhöhen. Ja
1.0667729377746582 Frage 3 Bei Bundestagswahlen sollen auch Jugendliche ab 16 Jahren wählen dürfen. Ja
1.0819740295410156 Frage 4 Die Förderung von Windenergie soll beendet werden. Nein
1.2291231155395508 Frage 5 Die Möglichkeiten der Vermieterinnen und Vermieter, Wohnungsmieten zu erhöhen, sollen gesetzlich stärker begrenzt werden. Ja
1.1265459060668945 Frage 6 Impfstoffe gegen Covid-19 sollen weiterhin durch Patente geschützt sein. Nein
1.0325229167938232 Frage 7 Der für das Jahr 2038 geplante Ausstieg aus der Kohleverstromung soll vorgezogen werden. Ja
1.1162502765655518 Frage 8 Alle Erwerbstätigen sollen in der gesetzlichen Rentenversicherung versichert sein müssen. Ja
1.1265528202056885 Frage 9 Das Recht anerkannter Flüchtlinge auf Familiennachzug soll abgeschafft werden. Nein
1.1248831748962402 Frage 10 Auf d